In [ ]:
# default_exp config

# Config

> This module manages configuration of the `planetarypy` package.

In [ ]:
# hide
from nbdev.showdoc import show_doc

In [ ]:
# export
import os
import shutil
from functools import reduce
from importlib.resources import path as resource_path
from pathlib import Path

import strictyaml as yaml

In [ ]:
# export
class Config:
    """Manage config stuff.

    Attributes
    -------
    path: pathlib.Path

    The key, value pairs found in the config file become attributes of the
    class instance after initialization.
    At minimum, there should be the `archive_path` attribute for storing data
    for this package.
    """

    # This enables a config path location override using env PYCISS_CONFIG
    fname = "planetarypy_config.yaml"
    path = Path(os.getenv("PLANETARYPY_CONFIG", Path.home() / f".{fname}"))

    def __init__(self, other_config=None):
        "Switch to other config file location with `other_config`."
        if other_config is not None:
            self.path = Path(other_config)
        if not self.path.exists():
            with resource_path("planetarypy.data", self.fname) as p:
                shutil.copy(p, self.path)
        self.read_config()

    def read_config(self):
        """Read the configfile and store config dict.

        If found, load config via `yaml` and store YAML dict as `d`.
        `storage_root` will be stored as attribute.
        """
        self.d = yaml.load(self.path.read_text())
        if not self.d["storage_root"].data:
            self.ask_storage_root()
        else:
            self.storage_root = Path(self.d["storage_root"].data)

    def get_value(self, key):
        """Get sub-dictionary by nested key.

        Parameters
        ----------
        nested_key: str
            A nested key in dotted format, e.g. cassini.uvis.indexes
        """
        return reduce(lambda c, k: c[k], key.split("."), self.d)

    def set_value(self, nested_key, value):
        """Set sub-dic using dotted key.

        Parameters
        ----------
        key: str
            A nested key in dotted format, e.g. cassini.uvis.ring_summary
        value: convertable to string
            Value for the given key to be stored.
        """
        dic = self.d
        keys = nested_key.split(".")
        for key in keys[:-1]:
            dic = dic[key]
        dic[keys[-1]] = value
        self.save()

    def save(self):
        "Write the YAML dict to file."
        with self.path.open("w") as f:
            f.write(self.d.as_yaml())

    def ask_storage_root(self):
        """Use input() to ask user for the storage_root path.

        The path will be stored in the YAML-dict and saved into existing config file
        at `Class.path`, either default or as given during init.
        `storage_root` attribute is set as well.
        """
        path = input(
            "Provide the root storage path where all downloaded and produced data will be stored:"
        )
        self.d["storage_root"] = path
        self.storage_root = Path(path)
        self.save()

## The Config() object

The `config` module instantiates a `config` object from the Config class.
Its attributes can be used to access several aspects relevant to the configuration of `planetarypy`.
Using an object approach enables easy growth and nesting over time.

> Note: Any cell that starts with an `# export` becomes part of the library.
  Any other cells become automatically part of tests.
  `exports` also adds the exported code to the docs.
  
> Note: A good "First issue" for collaborators would be to improve the Config class to be able to merge a new larger index catalog with an existing one the user might have configured. This will need to take care of existing timestamps and possible other indexes that were put in by the user.

In [ ]:
config = Config(other_config='test_config.yaml')

In [ ]:
config.storage_root

Path('/home/maye/big_data/planetary_data')

In [ ]:
show_doc(Config.read_config)

<h4 id="Config.read_config" class="doc_header"><code>Config.read_config</code><a href="__main__.py#L28" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.read_config</code>()

```
Read the configfile and store config dict.

If found, load config via `yaml` and store YAML dict as `d`.
`storage_root` will be stored as attribute.
```

In [ ]:
assert isinstance(config.d, yaml.YAML)

In [ ]:
assert isinstance(config.d.data, dict)

In [ ]:
show_doc(Config.get_config_value)

<h4 id="Config.get_config_value" class="doc_header"><code>Config.get_config_value</code><a href="__main__.py#L40" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.get_config_value</code>(**`key`**)

```
Get sub-dictionary by nested key.

Parameters
----------
nested_key: str
    A nested key in dotted format, e.g. cassini.uvis.indexes
```

In [ ]:
index = 'cassini.iss.indexes.moon_summary'

In [ ]:
config.get_config_value(index)

YAML({'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl', 'timestamp': '2021-06-11T19:21:04.233582'})

In [ ]:
show_doc(Config.set_config_value)

<h4 id="Config.set_config_value" class="doc_header"><code>Config.set_config_value</code><a href="__main__.py#L50" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.set_config_value</code>(**`nested_key`**, **`value`**)

```
Set sub-dic using dotted key.

Parameters
----------
key: str
    A nested key in dotted format, e.g. cassini.uvis.ring_summary
value: convertable to string
    Value for the given key to be stored.
```

In [ ]:
from datetime import datetime as dt
now = dt.now().isoformat()
now

'2021-06-11T19:24:38.164122'

In [ ]:
config.set_config_value('cassini.iss.indexes.moon_summary.timestamp', now)

In [ ]:
show_doc(Config.save)

<h4 id="Config.save" class="doc_header"><code>Config.save</code><a href="__main__.py#L67" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.save</code>()

```
Write the YAML dict to file.
```

In [ ]:
config.get_config_value(index)

YAML({'url': 'https://pds-rings.seti.org/holdings/metadata/COISS_2xxx/COISS_2999/COISS_2999_moon_summary.lbl', 'timestamp': '2021-06-11T19:24:38.164122'})

In [ ]:
show_doc(Config.ask_storage_root)

<h4 id="Config.ask_storage_root" class="doc_header"><code>Config.ask_storage_root</code><a href="__main__.py#L72" class="source_link" style="float:right">[source]</a></h4>

> <code>Config.ask_storage_root</code>()

```
Use input() to ask user for the storage_root path.

The path will be stored in the YAML-dict and saved into existing config file
at `Class.path`, either default or as given during init.
`storage_root` attribute is set as well.
```